In [36]:
import polars as pl
from sklearn import preprocessing 
import numpy as np

In [3]:
df = pl.read_csv("Data/training_set_features.csv")

In [4]:
labels = pl.read_csv("Data/training_set_labels.csv")

# training set features

In [5]:
df.shape

(26707, 36)

In [6]:
df.n_unique()

26707

In [7]:
df.describe()

statistic,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,f64,f64,str,str
"""count""",26707.0,26615.0,26591.0,26636.0,26499.0,26688.0,26665.0,26620.0,26625.0,26579.0,24547.0,24547.0,25736.0,25887.0,25903.0,14433.0,26316.0,26319.0,26312.0,26245.0,26193.0,26170.0,"""26707""","""25300""","""26707""","""26707""","""22284""","""25299""","""24665""","""25244""","""26707""","""26707""",26458.0,26458.0,"""13377""","""13237"""
"""null_count""",0.0,92.0,116.0,71.0,208.0,19.0,42.0,87.0,82.0,128.0,2160.0,2160.0,971.0,820.0,804.0,12274.0,391.0,388.0,395.0,462.0,514.0,537.0,"""0""","""1407""","""0""","""0""","""4423""","""1408""","""2042""","""1463""","""0""","""0""",249.0,249.0,"""13330""","""13470"""
"""mean""",13353.0,1.618486,1.262532,0.048844,0.725612,0.068982,0.825614,0.35864,0.337315,0.677264,0.220312,0.329735,0.283261,0.08259,0.111918,0.87972,3.850623,2.342566,2.35767,4.025986,2.719162,2.118112,null,null,null,null,null,null,null,null,null,null,0.886499,0.534583,null,null
"""std""",7709.791156,0.910311,0.618149,0.215545,0.446214,0.253429,0.379448,0.47961,0.472802,0.467531,0.414466,0.470126,0.450591,0.275266,0.315271,0.3253,1.007436,1.285539,1.362766,1.086565,1.385055,1.33295,null,null,null,null,null,null,null,null,null,null,0.753422,0.928173,null,null
"""min""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,"""18 - 34 Years""","""12 Years""","""Black""","""Female""","""<= $75,000, Above Poverty""","""Married""","""Own""","""Employed""","""atmpeygn""","""MSA, Not Principle City""",0.0,0.0,"""arjwrbjb""","""bxpfxfdn"""
"""25%""",6677.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,1.0,4.0,2.0,1.0,null,null,null,null,null,null,null,null,null,null,0.0,0.0,null,null
"""50%""",13353.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,2.0,2.0,4.0,2.0,2.0,null,null,null,null,null,null,null,null,null,null,1.0,0.0,null,null
"""75%""",20030.0,2.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,5.0,4.0,4.0,5.0,4.0,4.0,null,null,null,null,null,null,null,null,null,null,1.0,1.0,null,null
"""max""",26706.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,5.0,5.0,5.0,5.0,5.0,"""65+ Years""","""Some College""","""White""","""Male""","""Below Poverty""","""Not Married""","""Rent""","""Unemployed""","""qufhixun""","""Non-MSA""",3.0,3.0,"""xqicxuve""","""xzmlyyjv"""


In [8]:
df.null_count()

respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,92,116,71,208,19,42,87,82,128,2160,2160,971,820,804,12274,391,388,395,462,514,537,0,1407,0,0,4423,1408,2042,1463,0,0,249,249,13330,13470


So every column has some N/A values, exception: age_group, race, sex, hhs_geo_region (residence region), census_msa

A significant number amount of rows miss information about health insurance, employment industry and employment occupation.

In [9]:
df.select(pl.col("h1n1_concern").arr.explode()).unique()

h1n1_concern
i64
3
null
0
1
2


In [10]:
# check what the categories in each column are, for some columns such as age group the categories are not described
for col in df.columns:
    print(df.select(pl.col(col).arr.explode()).unique())

shape: (26_707, 1)
┌───────────────┐
│ respondent_id │
│ ---           │
│ i64           │
╞═══════════════╡
│ 17715         │
│ 23613         │
│ 24381         │
│ 399           │
│ 22297         │
│ …             │
│ 1953          │
│ 9155          │
│ 18203         │
│ 9807          │
│ 23324         │
└───────────────┘
shape: (5, 1)
┌──────────────┐
│ h1n1_concern │
│ ---          │
│ i64          │
╞══════════════╡
│ 3            │
│ 0            │
│ null         │
│ 1            │
│ 2            │
└──────────────┘
shape: (4, 1)
┌────────────────┐
│ h1n1_knowledge │
│ ---            │
│ i64            │
╞════════════════╡
│ null           │
│ 0              │
│ 1              │
│ 2              │
└────────────────┘
shape: (3, 1)
┌───────────────────────────┐
│ behavioral_antiviral_meds │
│ ---                       │
│ i64                       │
╞═══════════════════════════╡
│ null                      │
│ 0                         │
│ 1                         │
└───────────────

In [11]:
# create a variable for each column (useful in visual code)
'''
for col in df.columns:
    vars()[col] = df.select(pl.col(col).arr.explode()).unique()
'''

'\nfor col in df.columns:\n    vars()[col] = df.select(pl.col(col).arr.explode()).unique()\n'

In [24]:
# let's find out how many rows remain after removing N/A values in any column
print('number of remaining rows: {}'.format(df.drop_nulls().shape[0]))
print('remaining rows: {Wert:.2f} %'.format(Wert = df.drop_nulls().shape[0] / df.shape[0] * 100))

number of remaining rows: 6437
remaining rows: 24.10 %


<div class="alert alert-block alert-info">
<b>Dropping rows with null values is not an option here.</b>
</div>

In [35]:
# let's find out how many rows remain after removing N/A values in any column, WHEN HEALTH INSURANCE, EMPLOYMENT INDUSTRY AND EMPLOYMENT OCCUPATION ARE IGNORED
print('number of remaining rows: {}'.format(df.drop('health_insurance', 'employment_industry', 'employment_occupation').drop_nulls().shape[0]))
print('remaining rows: {Wert:.2f} %'.format(Wert = df.drop('health_insurance', 'employment_industry', 'employment_occupation').drop_nulls().shape[0] / df.shape[0] * 100))

number of remaining rows: 19642
remaining rows: 73.55 %


<div class="alert alert-block alert-info">
<b>Dropping rows with null values, but without considering health insurance, employment industry and occupation might work.</b>
</div>

# Training set labels

In [13]:
labels.shape

(26707, 3)

In [14]:
labels.n_unique()

26707

In [15]:
labels.describe()

statistic,respondent_id,h1n1_vaccine,seasonal_vaccine
str,f64,f64,f64
"""count""",26707.0,26707.0,26707.0
"""null_count""",0.0,0.0,0.0
"""mean""",13353.0,0.212454,0.465608
"""std""",7709.791156,0.409052,0.498825
"""min""",0.0,0.0,0.0
"""25%""",6677.0,0.0,0.0
"""50%""",13353.0,0.0,0.0
"""75%""",20030.0,0.0,1.0
"""max""",26706.0,1.0,1.0


No missinf data and values are either 0 or 1 (100 % probability)

In [29]:
fts = labels.join(df, on="respondent_id", how="inner")

In [30]:
# check shape, in case some rows are missing
fts.shape

(26707, 38)